In [188]:
#from google.colab import files
#uploaded = files.upload()

In [189]:
# !unzip archive.zip -d /content/extracted_data1
# !unzip archive\ \(1\).zip -d /content/extracted_data2
# !unzip archive\ \(2\).zip -d /content/extracted_data3

Streaming output truncated to the last 5000 lines.
  inflating: /content/extracted_data1/train/surprise/Training_41859921.jpg  
  inflating: /content/extracted_data1/train/surprise/Training_41882557.jpg  
  inflating: /content/extracted_data1/train/surprise/Training_4188746.jpg  
  inflating: /content/extracted_data1/train/surprise/Training_41892633.jpg  
  inflating: /content/extracted_data1/train/surprise/Training_41931319.jpg  
  inflating: /content/extracted_data1/train/surprise/Training_42011223.jpg  
  inflating: /content/extracted_data1/train/surprise/Training_42064379.jpg  
  inflating: /content/extracted_data1/train/surprise/Training_42079843.jpg  
  inflating: /content/extracted_data1/train/surprise/Training_42095143.jpg  
  inflating: /content/extracted_data1/train/surprise/Training_42106357.jpg  
  inflating: /content/extracted_data1/train/surprise/Training_42107103.jpg  
  inflating: /content/extracted_data1/train/surprise/Training_42143192.jpg  
  inflating: /content/extr

In [190]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [191]:
!cp /content/drive/MyDrive/data /content/ -r

In [192]:
#!rm -rf /content/cache/

In [193]:
# Install all required packages for enhanced data cleaning
!pip install lz4
!pip install insightface
!pip install mtcnn
!pip install kaggle
!pip install albumentations

print("✓ All packages installed")

In [ ]:
# Upload Kaggle API credentials (kaggle.json)
# Get your kaggle.json from https://www.kaggle.com/account

from google.colab import files
import os

print("Please upload your kaggle.json file:")
uploaded = files.upload()

# Create .kaggle directory and move the file
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("✓ Kaggle API configured")

In [ ]:
# Download FER2013 dataset from Kaggle automatically
print("="*60)
print("DOWNLOADING FER2013 FROM KAGGLE")
print("="*60)

# Download FER2013 dataset
!kaggle datasets download -d msambare/fer2013 -p /content/data --unzip

# Also download enhanced FER2013+ for additional quality data
print("\nDownloading enhanced FER2013+ dataset...")
!kaggle datasets download -d ashishpatel26/facial-expression-recognitionferchallenge -p /content/data/fer2013_plus --unzip

print("\n✓ FER2013 datasets downloaded successfully")
print("\nAvailable data files:")
!ls -lh /content/data/

In [ ]:
# Upload our custom data cleaning and Kaggle fetcher modules
# These modules provide automated data cleaning functionality

print("Upload the following files:")
print("  - kaggle_data_fetcher.py")
print("  - data_cleaner.py")
print()

from google.colab import files
uploaded = files.upload()

print("\n✓ Modules uploaded successfully")

In [194]:
import lz4
import joblib
print("lz4 version:", lz4.__version__)
print("joblib version:", joblib.__version__)


lz4 version: 4.4.5
joblib version: 1.5.2


In [195]:
# ===============================
# 1) Imports + Seeds
# ===============================
import os
import cv2
import numpy as np
import pandas as pd
from glob import glob
import tensorflow as tf
import albumentations as A
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics.pairwise import cosine_distances
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

np.random.seed(42)
tf.random.set_seed(42)
print("TensorFlow:", tf.__version__)


TensorFlow: 2.19.0


In [196]:
from joblib import Memory
memory = Memory(location="cache", compress=True)


In [197]:
# ===============================
# 2) Preprocessing Function
# ===============================
def preprocess_image(img):
    """Preprocess single grayscale image"""
    img = cv2.resize(img, (48, 48))
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img = clahe.apply(img)
    img = img.astype('float32') / 255.0
    return img[..., np.newaxis]


def pixels_to_img(pixel_str):
    """Convert pixel string to image array"""
    return np.fromstring(pixel_str, dtype=np.uint8, sep=' ').reshape(48, 48)


print("✓ Preprocessing functions defined")


✓ Preprocessing functions defined


In [198]:
def is_blur_ok(img, threshold=80):
    """Check if image has acceptable sharpness"""
    laplacian_var = cv2.Laplacian(img, cv2.CV_64F).var()
    return laplacian_var > threshold


def is_brightness_ok(img, low=40, high=220):
    """Check if image has acceptable brightness"""
    mean_brightness = img.mean()
    return low < mean_brightness < high


print("✓ Quality filter functions defined")

✓ Quality filter functions defined


In [199]:
def init_face_detector():
    """Initialize MTCNN face detector"""
    try:
        from mtcnn import MTCNN

        detector = MTCNN()
        print("✓ MTCNN face detector loaded")
        return detector
    except ImportError:
        print("⚠ MTCNN not available. Install with: pip install mtcnn")
        return None


def has_valid_face(img, detector, min_face_ratio=0.30):
    """Check if image contains exactly one valid face"""
    if detector is None:
        return True  # Skip face detection if not available

    rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    faces = detector.detect_faces(rgb)

    if len(faces) != 1:
        return False

    x, y, w, h = faces[0]['box']
    face_area = w * h
    img_area = 48 * 48

    return (face_area / img_area) > min_face_ratio


print("✓ Face detection functions defined")

✓ Face detection functions defined


In [200]:
def init_embedding_model():
    """Initialize InsightFace embedding model"""
    try:
        from insightface.app import FaceAnalysis
        app = FaceAnalysis(name="buffalo_l", providers=["CPUExecutionProvider"])
        app.prepare(ctx_id=0)
        print("✓ InsightFace embedding model loaded")
        return app
    except ImportError:
        print("⚠ InsightFace not available. Install with: pip install insightface")
        return None
    except Exception as e:
        print(f"⚠ Could not load InsightFace: {e}")
        return None


def get_face_embedding(img, embedding_model):
    """Extract face embedding from image"""
    if embedding_model is None:
        return None

    rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    faces = embedding_model.get(rgb)

    if len(faces) != 1:
        return None

    return faces[0].embedding


print("✓ Face embedding functions defined")

✓ Face embedding functions defined


In [201]:
def clean_fer2013(csv_path, use_face_detection=True, use_embeddings=True):
    """
    Load and clean FER2013 dataset with quality filters

    Args:
        csv_path: Path to FER2013 CSV file
        use_face_detection: Whether to use MTCNN face detection
        use_embeddings: Whether to use embedding-based outlier removal
    """
    import lz4
    import joblib
    if not os.path.exists(csv_path):
        print(f"Error: FER2013 CSV not found at {csv_path}")
        return np.array([]), np.array([])

    print(f"\n{'='*60}")
    print(f"Loading FER2013 from {csv_path}")
    print(f"{'='*60}")

    df = pd.read_csv(csv_path)

    # Handle column naming variations
    if ' pixels' in df.columns:
        df.rename(columns={' pixels': 'pixels'}, inplace=True)
    if ' emotion' in df.columns:
        df.rename(columns={' emotion': 'emotion'}, inplace=True)

    print(f"Original dataset size: {len(df)} images")

    # Initialize detectors
    face_detector = init_face_detector() if use_face_detection else None
    embedding_model = init_embedding_model() if use_embeddings else None

    # Step 1: Quality filtering
    print(f"\n[Step 1/3] Applying quality filters...")
    clean_rows = []
    embeddings = []

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Filtering"):
        try:
            img = pixels_to_img(row['pixels'])

            # Quality checks
            if not is_blur_ok(img):
                continue

            if not is_brightness_ok(img):
                continue

            # Face detection check
            if use_face_detection and not has_valid_face(img, face_detector):
                continue

            # Extract embedding
            if use_embeddings:
                emb = get_face_embedding(img, embedding_model)
                if emb is None:
                    continue
                embeddings.append(emb)

            clean_rows.append(row)

        except Exception as e:
            continue

    clean_df = pd.DataFrame(clean_rows).reset_index(drop=True)
    print(f"After quality filtering: {len(clean_df)} images ({len(clean_df)/len(df)*100:.1f}%)")

    # Step 2: Embedding-based outlier removal
    if use_embeddings and len(embeddings) > 0:
        print(f"\n[Step 2/3] Removing embedding outliers...")
        clean_df['embedding'] = embeddings

        def remove_outliers(group):
            if len(group) < 10:
                return group

            E = np.stack(group['embedding'].values)
            center = E.mean(axis=0)
            distances = cosine_distances(E, center.reshape(1, -1)).flatten()
            threshold = distances.mean() + 2.5 * distances.std()

            return group[distances < threshold]

        clean_df = clean_df.groupby('emotion', group_keys=False).apply(remove_outliers)
        clean_df = clean_df.drop(columns=['embedding']).reset_index(drop=True)

        print(f"After outlier removal: {len(clean_df)} images ({len(clean_df)/len(df)*100:.1f}%)")

    # Step 3: Convert to training tensors
    print(f"\n[Step 3/3] Converting to tensors...")
    X, y = [], []

    for idx, row in clean_df.iterrows():
        try:
            img = pixels_to_img(row['pixels'])
            img = img.astype('float32') / 255.0
            label = min(int(row['emotion']), 6)

            X.append(img[..., np.newaxis])
            y.append(label)
        except Exception as e:
            continue

    X = np.array(X)
    y = to_categorical(y, num_classes=7)

    print(f"\n✓ FER2013 cleaned: {X.shape}")
    print(f"  Kept {len(X)}/{len(df)} images ({len(X)/len(df)*100:.1f}%)\n")

    return X, y


print("✓ FER2013 cleaning function defined")

✓ FER2013 cleaning function defined


In [202]:
import lz4
import joblib
print("lz4 version:", lz4.__version__)
print("joblib version:", joblib.__version__)


lz4 version: 4.4.5
joblib version: 1.5.2


In [203]:
def load_ck_plus(csv_path):
    """Load CK+ dataset from CSV"""
    if not os.path.exists(csv_path):
        print(f"⚠ CK+ CSV not found at {csv_path}")
        return np.array([]), np.array([])

    df = pd.read_csv(csv_path)

    # Handle column naming
    if ' pixels' in df.columns:
        df.rename(columns={' pixels': 'pixels'}, inplace=True)
    if ' emotion' in df.columns:
        df.rename(columns={' emotion': 'emotion'}, inplace=True)

    df = df[df['emotion'].isin([0, 1, 2, 3, 4, 5, 6])]

    X, y = [], []
    for idx, row in df.iterrows():
        try:
            pixels = row['pixels']
            label = min(int(row['emotion']), 6)
            img = np.fromstring(pixels, dtype=np.uint8, sep=' ').reshape(48, 48)
            img = img.astype('float32') / 255.0
            X.append(img[..., np.newaxis])
            y.append(label)
        except:
            continue

    X = np.array(X)
    y = to_categorical(y, num_classes=7)
    print(f"✓ CK+ loaded: {X.shape}")
    return X, y


# def load_raf_db(root_path):
#     """Load RAF-DB dataset from image folders"""
#     if not os.path.exists(root_path):
#         print(f"⚠ RAF-DB not found at {root_path}")
#         return np.array([]), np.array([])

#     X, y = [], []
#     emotion_map = {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6}

#     for split in ['train', 'test']:
#         split_path = os.path.join(root_path, split)
#         if not os.path.exists(split_path):
#             continue

#         for filename in os.listdir(split_path):
#             if not filename.endswith(('.jpg', '.png')):
#                 continue

#             try:
#                 img_path = os.path.join(split_path, filename)
#                 img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
#                 if img is None:
#                     continue

#                 label = int(filename.split('_')[0])
#                 label = emotion_map.get(label)
#                 if label is None:
#                     continue

#                 img = cv2.resize(img, (48, 48)).astype('float32') / 255.0
#                 X.append(img[..., np.newaxis])
#                 y.append(label)
#             except:
#                 continue

#     X = np.array(X)
#     y = to_categorical(y, num_classes=7)
#     print(f"✓ RAF-DB loaded: {X.shape}")
#     return X, y

def load_raf_db(root_path):
    """
    Load RAF-DB dataset from image folders (Kaggle/ShuvoAlok structure).
    Returns image data (X) and integer labels (y, 0-6).
    """
    if not os.path.exists(root_path):
        print(f"⚠ RAF-DB not found at {root_path}")
        return np.array([]), np.array([])

    X, y = [], []
    # This map converts the folder name (1-7 as strings) into sequential
    # zero-indexed integer labels (0-6).
    emotion_map = {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}

    for split in ['train', 'test']:
        split_path = os.path.join(root_path, split)
        if not os.path.exists(split_path):
            continue

        # Iterate over the emotion subdirectories (e.g., '1', '2', '3', etc.)
        for emotion_folder in os.listdir(split_path):
            label_str = emotion_folder

            # Use the folder name as the label index
            label = emotion_map.get(label_str)

            if label is None:
                # Skip non-emotion folders
                continue

            emotion_path = os.path.join(split_path, emotion_folder)

            if not os.path.isdir(emotion_path):
                continue

            # Iterate over the image files inside the emotion folder
            for filename in os.listdir(emotion_path):
                # print(filename)
                if not filename.endswith(('.jpg', '.png', '.jpeg')):
                    continue

                try:
                    img_path = os.path.join(emotion_path, filename)
                    # Load image as grayscale
                    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

                    if img is None:
                        continue

                    # Resize and normalize
                    img = cv2.resize(img, (48, 48)).astype('float32') / 255.0

                    # Add channel dimension for CNN input (48, 48, 1)
                    X.append(img[..., np.newaxis])
                    y.append(label)

                except Exception:
                    # Catch and skip problematic files
                    continue

    X = np.array(X)
    y = np.array(y) # The labels are now returned as a simple NumPy array of integers (0-6)

    # Example output shape (e.g., if 2000 images were loaded):
    # X.shape will be (2000, 48, 48, 1)
    # y.shape will be (2000,)
    from tensorflow.keras.utils import to_categorical # Assuming this is available globally
    y = to_categorical(y, num_classes=7)
    print(f"✓ RAF-DB loaded: X shape {X.shape}, y shape {y.shape}")
    return X, y

# def load_kdef(root_path):
#     """Load KDEF dataset from image folders"""
#     if not os.path.exists(root_path):
#         print(f"⚠ KDEF not found at {root_path}")
#         return np.array([]), np.array([])

#     X, y = [], []

#     for label in range(7):
#         label_path = os.path.join(root_path, str(label))
#         if not os.path.exists(label_path):
#             continue

#         for filename in os.listdir(label_path):
#             if not filename.endswith(('.jpg', '.png')):
#                 continue

#             try:
#                 img_path = os.path.join(label_path, filename)
#                 img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
#                 if img is None:
#                     continue

#                 img = cv2.resize(img, (48, 48)).astype('float32') / 255.0
#                 X.append(img[..., np.newaxis])
#                 y.append(label)
#             except:
#                 continue

#     X = np.array(X)
#     y = to_categorical(y, num_classes=7)
#     print(f"✓ KDEF loaded: {X.shape}")
#     return X, y

def load_kdef(root_path):
    """
    Load KDEF dataset from image folders (Assumes 'emotion_name' folders directly under root_path).
    Returns image data (X) and one-hot encoded labels (y).
    """
    if not os.path.exists(root_path):
        print(f"⚠ KDEF not found at {root_path}")
        return np.array([]), np.array([])

    X, y = [], []
    # This map converts the folder name into sequential zero-indexed integer labels (0-6).
    emotion_map = {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}

    # Iterate directly over the emotion subdirectories under the root_path
    for emotion_folder in os.listdir(root_path):
        label_str = emotion_folder

        # Get the integer label from the emotion name
        label = emotion_map.get(label_str)

        if label is None:
            # Skip folders that are not one of the seven emotions
            continue

        emotion_path = os.path.join(root_path, emotion_folder)

        if not os.path.isdir(emotion_path):
            continue

        # Iterate over the image files inside the emotion folder
        for filename in os.listdir(emotion_path):
            # Check for common image extensions
            if not filename.endswith(('.jpg', '.png', '.jpeg', '.bmp', '.tif')):
                continue

            try:
                img_path = os.path.join(emotion_path, filename)
                # Load image as grayscale
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

                if img is None:
                    continue

                # Resize and normalize
                img = cv2.resize(img, (48, 48)).astype('float32') / 255.0

                # Add channel dimension for CNN input (48, 48, 1)
                X.append(img[..., np.newaxis])
                y.append(label) # Append the integer label

            except Exception:
                # Catch and skip problematic files
                continue

    X = np.array(X)
    y = np.array(y)

    # Convert integer labels to one-hot encoded vectors, consistent with CK+
    # Ensure 'to_categorical' is imported/available in your environment
    y = to_categorical(y, num_classes=7)

    print(f"✓ KDEF loaded: X shape {X.shape}, y shape {y.shape}")
    return X, y

print("✓ Dataset loader functions defined")

✓ Dataset loader functions defined


In [204]:
# ===============================
# DATA LOADING WITH KAGGLE INTEGRATION
# ===============================
# FER2013 is the PRIMARY dataset - we always use it
# Additional datasets from Kaggle enhance model performance

USE_FER_CLEANING = True     # ALWAYS use FER2013 with cleaning
USE_FACE_DETECTION = False  # Set to True for stricter face validation
USE_EMBEDDINGS = True       # Use embedding-based outlier removal
USE_KAGGLE_AUTO = True      # Use automated Kaggle data fetching

print("="*60)
print("LOADING DATASETS (FER2013 PRIMARY)")
print("="*60)

# Load FER2013 - this is our core dataset
if USE_FER_CLEANING:
    print("\n📊 Loading FER2013 (PRIMARY DATASET)...")
    X_fer, y_fer = clean_fer2013(
        "/content/data/fer2013/fer2013.csv",  # Main FER2013 from Kaggle
        use_face_detection=USE_FACE_DETECTION,
        use_embeddings=USE_EMBEDDINGS
    )
else:
    # Fallback: load without cleaning (not recommended)
    print("\n📊 Loading FER2013 without cleaning...")
    df = pd.read_csv("/content/data/fer2013/fer2013.csv")
    X_fer = np.array([pixels_to_img(p) for p in df['pixels'].values])
    X_fer = X_fer.astype('float32') / 255.0
    X_fer = X_fer[..., np.newaxis]
    y_fer = to_categorical(df['emotion'].values, num_classes=7)
    print(f"✓ FER2013 loaded: {X_fer.shape}")

# Optional: Load additional datasets to enhance FER2013
print("\n📊 Loading supplementary datasets...")

# CK+ dataset (if available)
try:
    X_ck, y_ck = load_ck_plus("/content/data/ck_plus/ckplus.csv")
except:
    X_ck, y_ck = np.array([]), np.array([])
    print("⚠️  CK+ not available (optional)")

# RAF-DB dataset (if available)
try:
    X_raf, y_raf = load_raf_db("/content/data/raf_db")
except:
    X_raf, y_raf = np.array([]), np.array([])
    print("⚠️  RAF-DB not available (optional)")

# KDEF dataset (if available)
try:
    X_kdef, y_kdef = load_kdef("/content/data/kdef")
except:
    X_kdef, y_kdef = np.array([]), np.array([])
    print("⚠️  KDEF not available (optional)")

print("\n" + "="*60)
print("DATASET SUMMARY")
print("="*60)


LOADING ALL DATASETS

Loading FER2013 from /content/data/icml_face_data.csv
Original dataset size: 35887 images
⚠ InsightFace not available. Install with: pip install insightface

[Step 1/3] Applying quality filters...


Filtering: 100%|██████████| 35887/35887 [00:21<00:00, 1708.11it/s]


After quality filtering: 0 images (0.0%)

[Step 3/3] Converting to tensors...

✓ FER2013 cleaned: (0,)
  Kept 0/35887 images (0.0%)

✓ CK+ loaded: (902, 48, 48, 1)
✓ RAF-DB loaded: X shape (35887, 48, 48, 1), y shape (35887, 7)
✓ KDEF loaded: X shape (2938, 48, 48, 1), y shape (2938, 7)

DATASET LOADING COMPLETE


In [205]:
datasets = []
if len(X_fer) > 0:
    datasets.append((X_fer, y_fer))
    print(f"✓ FER2013: {X_fer.shape}")
if len(X_ck) > 0:
    datasets.append((X_ck, y_ck))
    print(f"✓ CK+: {X_ck.shape}")
if len(X_raf) > 0:
    datasets.append((X_raf, y_raf))
    print(f"✓ RAF-DB: {X_raf.shape}")
if len(X_kdef) > 0:
    datasets.append((X_kdef, y_kdef))
    print(f"✓ KDEF: {X_kdef.shape}")

if not datasets:
    raise ValueError("❌ No datasets loaded!")

X = np.concatenate([d[0] for d in datasets], axis=0)
y = np.concatenate([d[1] for d in datasets], axis=0)

print(f"\n{'='*60}")
print(f"TOTAL COMBINED DATA: {X.shape}")
print(f"{'='*60}\n")

# Split data
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=np.argmax(y, axis=1)
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42,
    stratify=np.argmax(y_temp, axis=1)
)

print(f"📊 Train: {X_train.shape}")
print(f"📊 Val:   {X_val.shape}")
print(f"📊 Test:  {X_test.shape}")

✓ CK+: (902, 48, 48, 1)
✓ RAF-DB: (35887, 48, 48, 1)
✓ KDEF: (2938, 48, 48, 1)

TOTAL COMBINED DATA: (39727, 48, 48, 1)

📊 Train: (31781, 48, 48, 1)
📊 Val:   (3973, 48, 48, 1)
📊 Test:  (3973, 48, 48, 1)


In [206]:
train_aug = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.7),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5)
])


class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, X, y, batch_size=64, augment=False):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.augment = augment
        self.indices = np.arange(len(X))

    def __len__(self):
        return int(np.ceil(len(self.X) / self.batch_size))

    def __getitem__(self, idx):
        batch_idx = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_X = self.X[batch_idx]
        batch_y = self.y[batch_idx]

        if self.augment:
            aug_X = []
            for img in batch_X:
                img_uint8 = (img.squeeze() * 255).astype(np.uint8)
                aug_img = train_aug(image=img_uint8)['image']
                aug_X.append(aug_img[..., np.newaxis] / 255.0)
            batch_X = np.array(aug_X)

        return batch_X, batch_y

    def on_epoch_end(self):
        np.random.shuffle(self.indices)


# Create training generator
train_gen = DataGenerator(X_train, y_train, batch_size=64, augment=True)
print(f"✓ Data generator created with {len(train_gen)} batches")

✓ Data generator created with 497 batches


/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [207]:
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(np.argmax(y_train, axis=1)),
    y=np.argmax(y_train, axis=1)
)
class_weights = dict(enumerate(class_weights))

emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
print("Class weights:")
for i, (label, weight) in enumerate(zip(emotion_labels, class_weights.values())):
    print(f"  {label}: {weight:.3f}")

Class weights:
  Angry: 1.048
  Disgust: 5.530
  Fear: 1.020
  Happy: 0.599
  Sad: 0.854
  Surprise: 0.863
  Neutral: 1.132


In [208]:
def residual_block(x, filters):
    """Residual block with skip connection"""
    shortcut = x

    x = Conv2D(filters, 3, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters, 3, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)

    if shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, 1, padding='same')(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x


def create_emotion_model():
    """Create CNN model with residual blocks"""
    inputs = Input(shape=(48, 48, 1))

    x = Conv2D(32, 3, padding='same', kernel_initializer='he_normal')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = residual_block(x, 64)
    x = MaxPooling2D()(x)

    x = residual_block(x, 128)
    x = MaxPooling2D()(x)

    x = residual_block(x, 256)
    x = MaxPooling2D()(x)

    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)

    outputs = Dense(7, activation='softmax')(x)

    return Model(inputs=inputs, outputs=outputs)


# Create model
model = create_emotion_model()
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 48, 48, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 48, 48,    │        320 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 48, 48,    │        128 │ conv2d_10[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_7        │ (None, 48, 48,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 48, 48,    │     18,496 │ activation_7[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 48, 48,    │        256 │ conv2d_11[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_8        │ (None, 48, 48,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 48, 48,    │     36,928 │ activation_8[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 48, 48,    │        256 │ conv2d_12[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_13 (Conv2D)  │ (None, 48, 48,    │      2,112 │ activation_7[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 48, 48,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │ conv2d_13[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_9        │ (None, 48, 48,    │          0 │ add_3[0][0]       │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 24, 24,    │          0 │ activation_9[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_14 (Conv2D)  │ (None, 24, 24,    │     73,856 │ max_pooling2d_3[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 24, 24,    │        512 │ conv2d_14[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_10       │ (None, 24, 24,    │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_15 (Conv2D)  │ (None, 24, 24,    │    147,584 │ activation_10[0]

 Total params: 1,277,191 (4.87 MB)

 Trainable params: 1,275,335 (4.87 MB)

 Non-trainable params: 1,856 (7.25 KB)

In [209]:
model.compile(
    optimizer=Adam(learning_rate=3e-4),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

print("✓ Model compiled successfully")

✓ Model compiled successfully


In [210]:
callbacks = [
    ReduceLROnPlateau(
        monitor="val_loss",
        patience=7,
        factor=0.5,
        min_lr=1e-6,
        verbose=1
    ),
    EarlyStopping(
        monitor="val_loss",
        patience=25,
        min_delta=0.0005,
        mode="min",
        restore_best_weights=True
    )
]

print("✓ Callbacks configured")

✓ Callbacks configured


In [ ]:
print("\n" + "="*60)
print("STARTING TRAINING")
print("="*60 + "\n")

history = model.fit(
    train_gen,
    validation_data=(X_val, y_val),
    epochs=50,
    callbacks=callbacks,
    class_weight=class_weights,
    verbose=1
)

print("\n✓ Training complete!")


STARTING TRAINING

Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


497/497 ━━━━━━━━━━━━━━━━━━━━ 57s 65ms/step - accuracy: 0.1714 - loss: 2.0543 - val_accuracy: 0.2258 - val_loss: 1.8668 - learning_rate: 3.0000e-04
Epoch 2/50
497/497 ━━━━━━━━━━━━━━━━━━━━ 22s 45ms/step - accuracy: 0.2611 - loss: 1.8404 - val_accuracy: 0.3317 - val_loss: 1.7844 - learning_rate: 3.0000e-04
Epoch 3/50
497/497 ━━━━━━━━━━━━━━━━━━━━ 22s 45ms/step - accuracy: 0.3672 - loss: 1.6998 - val_accuracy: 0.4528 - val_loss: 1.5464 - learning_rate: 3.0000e-04
Epoch 4/50
497/497 ━━━━━━━━━━━━━━━━━━━━ 24s 47ms/step - accuracy: 0.4212 - loss: 1.6039 - val_accuracy: 0.4533 - val_loss: 1.5406 - learning_rate: 3.0000e-04
Epoch 5/50
497/497 ━━━━━━━━━━━━━━━━━━━━ 24s 49ms/step - accuracy: 0.4639 - loss: 1.5233 - val_accuracy: 0.4656 - val_loss: 1.5332 - learning_rate: 3.0000e-04
Epoch 6/50
497/497 ━━━━━━━━━━━━━━━━━━━━ 23s 46ms/step - accuracy: 0.4973 - loss: 1.4541 - val_accuracy: 0.4981 - val_loss: 1.4620 - learning_rate: 3.0000e-04
Epoch 7/50
497/497 ━━━━━━━━━━━━━━━━━━━━ 41s 46ms/step - accurac

In [ ]:
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
plt.plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.legend(fontsize=11)
plt.title('Model Accuracy', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss', linewidth=2)
plt.plot(history.history['val_loss'], label='Val Loss', linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend(fontsize=11)
plt.title('Model Loss', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n📊 Test Accuracy: {test_acc * 100:.2f}%")
print(f"📊 Test Loss: {test_loss:.4f}")

In [ ]:
def tta_predict(model, X, n_augmentations=5):
    """Apply Test Time Augmentation for better predictions"""
    print(f"Applying TTA with {n_augmentations} augmentations...")
    predictions = [model.predict(X, verbose=0)]

    for i in range(n_augmentations):
        aug_X = []
        for img in X:
            img_uint8 = (img.squeeze() * 255).astype(np.uint8)
            aug_img = train_aug(image=img_uint8)['image']
            aug_X.append(aug_img[..., np.newaxis] / 255.0)
        aug_X = np.array(aug_X)
        predictions.append(model.predict(aug_X, verbose=0))
        print(f"  Augmentation {i+1}/{n_augmentations} complete")

    return np.mean(predictions, axis=0)


tta_preds = tta_predict(model, X_test, n_augmentations=5)
y_pred = np.argmax(tta_preds, axis=1)
y_true = np.argmax(y_test, axis=1)
tta_acc = np.mean(y_pred == y_true)

print(f"\n✓ TTA complete!")

In [ ]:
print("\n" + "="*60)
print("FINAL RESULTS")
print("="*60)
print(f"Best Val Accuracy:       {max(history.history['val_accuracy']) * 100:.2f}%")
print(f"Test Accuracy:           {test_acc * 100:.2f}%")
print(f"Test Accuracy with TTA:  {tta_acc * 100:.2f}%")
print("="*60 + "\n")

In [ ]:
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=emotion_labels, yticklabels=emotion_labels,
            cbar_kws={'label': 'Count'})
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('True', fontsize=12)
plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Print classification report
print("\nClassification Report:")
print("="*60)
print(classification_report(y_true, y_pred, target_names=emotion_labels))

In [ ]:
model_filename = "emotion_model_cleaned.keras"
model.save(model_filename)
print(f"✓ Model saved as '{model_filename}'")

In [ ]:
# Download the trained model to your local machine
from google.colab import files

try:
    files.download(model_filename)
    print(f"✓ Download started for {model_filename}")
except:
    print(f"⚠ Could not download. File is available at: /content/{model_filename}")

In [ ]:
# Visualize sample predictions
fig, axes = plt.subplots(3, 5, figsize=(16, 10))
axes = axes.ravel()

np.random.seed(42)
sample_indices = np.random.choice(len(X_test), 15, replace=False)

for i, idx in enumerate(sample_indices):
    img = X_test[idx]
    true_label = emotion_labels[np.argmax(y_test[idx])]
    pred_label = emotion_labels[y_pred[idx]]
    confidence = tta_preds[idx][y_pred[idx]] * 100

    axes[i].imshow(img.squeeze(), cmap='gray')
    color = 'green' if true_label == pred_label else 'red'
    axes[i].set_title(
        f'True: {true_label}\nPred: {pred_label} ({confidence:.1f}%)',
        color=color,
        fontsize=10,
        fontweight='bold'
    )
    axes[i].axis('off')

plt.suptitle('Sample Predictions (Green=Correct, Red=Wrong)',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate per-class accuracy
print("Per-Class Accuracy:")
print("="*60)

for i, label in enumerate(emotion_labels):
    mask = y_true == i
    if mask.sum() > 0:
        class_acc = (y_pred[mask] == i).sum() / mask.sum() * 100
        print(f"{label:12s}: {class_acc:5.2f}% ({mask.sum()} samples)")
    else:
        print(f"{label:12s}: No samples in test set")

print("="*60)